## SKFlow Simple Linear Regression on Parkinson's Disease Telemonitoring Dataset

https://archive.ics.uci.edu/ml/datasets/Parkinsons+Telemonitoring

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.learn.python.learn as skflow
from sklearn import datasets, metrics
import numpy as np
import os.path
import pickle

### Function Defintions

In [ ]:
### Converting Data into Tensors
def convert_dataframe(dataframe, colnames, labelcol):
    X = np.array(dataframe[colnames].iloc[:,]).astype(np.float32)
    Y = np.array(dataframe[labelcol].iloc[:,]).astype(np.float32)  
    return X, Y

### load or generate random ids for test and training sets
def slice_ids(length,run_id):
    idfile = "models/"+run_id+"_ids"
    if os.path.isfile(idfile): 
        ### Re-run model
        with open(idfile, 'rb') as fp:
            randomInd = pickle.load(fp)
    else:
        ### First run
        randomInd = np.random.permutation(length)
        with open(idfile, 'wb') as fp:
            pickle.dump(randomInd, fp)
    
    ###  70% of ids randomly selected for training
    ###  30% of ids randomly selected for testing
    mid = int(.7*length)
    trainidx = randomInd[:mid]
    testidx = randomInd[mid:]
    return trainidx, testidx

def run_LinearRegressor(run_id, steps = 100):
    ### Get Training and Testing indexes
    trainidx, testidx = slice_ids(len(X),run_id)

    ### Define classifier - Simple Linear Regression
    ### model_dir - this is where the model is saved. To re-run use the same runIdentifier
    feature_columns = skflow.infer_real_valued_columns_from_input(X)
    model = skflow.LinearRegressor(
        feature_columns=feature_columns,
        model_dir="models/SLR/"+run_id,
        enable_centered_bias=False)
    
    model.fit(X[trainidx,:], 
              Y[trainidx],#.astype(np.int), 
              steps=steps)
    
    y_p = model.predict(X[testidx,:])
    y_t = Y[testidx]
    return np.linalg.norm(y_t-y_p)/y_t.shape[0]

### Load data and convert to tensors

In [ ]:
### Load datafile into dataframe
datafile = "data/data.txt"
df_data = pd.read_csv(datafile)

### Create list of column names for feature data.
featcol = list(df_data.columns.values)
featcol.remove("subject#")
featcol.remove("total_UPDRS")
featcol.remove("motor_UPDRS")

### First Model fitting Motor_UPDRS - SLR

In [ ]:
labelcol = "motor_UPDRS" 
X, Y = convert_dataframe(df_data, featcol, labelcol)

In [ ]:
%%time
print run_LinearRegressor("Motor_UPDRS_001", 10000)

In [ ]:
%%time
print run_LinearRegressor("Motor_UPDRS_002", 10000)

In [ ]:
%%time
print run_LinearRegressor("Motor_UPDRS_003", 10000)

In [ ]:
%%time
print run_LinearRegressor("Motor_UPDRS_004", 10000)

### Second Model fitting Total_UPDRS - SLR

In [ ]:
labelcol = "total_UPDRS"
X, Y = convert_dataframe(df_data, featcol, labelcol)

In [ ]:
%%time
print run_LinearRegressor("Total_UPDRS_001", 10000)

In [ ]:
%%time
print run_LinearRegressor("Total_UPDRS_002", 10000)

In [ ]:
%%time
print run_LinearRegressor("Total_UPDRS_003", 10000)

In [ ]:
%%time
print run_LinearRegressor("Total_UPDRS_004", 10000)